In [1]:
from db.db import backup_database

backup_destination = r"N:\PPDO\BS\Records Department\Archive_Data"
backup_database(backup_dir=backup_destination, compress=True)

'N:\\PPDO\\BS\\Records Department\\Archive_Data\\archives_backup_20250820_213750.sql.bz2'

In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import or_
import pandas as pd
import numpy as np
from db import get_db_engine
from db.models import FileLocation, File, FileEmbedding
from text_extraction.pdf_extraction import PDFTextExtractor
from embedding.minilm import MiniLMEmbedder
from utils import extract_server_dirs, build_file_path
server_mount = r"N:\PPDO\Records"
target_filepath = r"N:\PPDO\Records\23xx   Crown College (College 3)\2318\2318\H - Submittals\Lumber_ProductData.pdf"
target_location = r"N:\PPDO\Records\37xx   Oakes College\3701\3701"


pdf_extractor = PDFTextExtractor()
embedder = MiniLMEmbedder()

text = pdf_extractor(target_filepath)
embeddings = embedder.encode(text)
target_loc_dirs = extract_server_dirs(full_path=target_location, base_mount=server_mount)
engine = get_db_engine()

target_sim_distance = lambda file_embeds: 1 - np.dot(file_embeds, embeddings)
build_local_path = lambda row: build_file_path(base_mount=server_mount, server_dir=row['file_server_directories'], filename=row['filename'])

with Session(engine) as db_session:
    files_located_in_dir = or_(
        FileLocation.file_server_directories == target_loc_dirs,
        FileLocation.file_server_directories.startswith(target_loc_dirs + '/')
    )

    query = db_session.query(FileLocation, FileEmbedding)\
        .join(File, FileLocation.file)\
        .join(FileEmbedding, File.hash == FileEmbedding.file_hash)\
        .filter(files_located_in_dir)
    
    # query into dataframe
    df = pd.read_sql(query.statement, db_session.bind)
    df['similarity'] = df['minilm_emb'].apply(target_sim_distance)
    df = df.sort_values(by='similarity', ascending=True)

    # Add local file path
    df['local_filepath'] = df.apply(build_local_path, axis=1)

df.head()


